---

# Willem Bruin

This Jupyter notebook applies the **MCCQRNN** model to provided FreeSurfer (FS)-derived MRI data.

---

### Overview

This notebook performs the following steps:

* Prepares FreeSurfer-derived MRI features from the ENIGMA-ANXIETY working group
* Trains the MCCQRNN model on healthy controls using cross-validation
* Applies the trained model to unseen patients
* Computes uncertainty-adjusted Brain Age Gap (BAG) scores
* Generates plots to evaluate model performance
* Conducts occlusion sensitivity mapping

---

### Model Information

The MCCQRNN model is described in:

> Hahn, T., Ernsting, J., Winter, N. R., Holstein, V., Leenings, R., Beisemann, M., Fisch, L., Sarink, K., Emden, D., Opel, N., et al. (2022). *An uncertainty-aware, shareable, and transparent neural network architecture for brain-age modeling*. **Science Advances, 8**(1), eabg9471.
> [https://www.science.org/doi/10.1126/sciadv.abg9471](https://www.science.org/doi/10.1126/sciadv.abg9471)

The source code is available here:
[https://github.com/wwu-mmll/uncertainty-brain-age](https://github.com/wwu-mmll/uncertainty-brain-age)

---

### Implementation Notes

The version of the MCCQRNN model used in this project is copied to `/Scripts/MCCQRNN_Regressor.py` for usage.

---


In [ ]:
import os
import shutil
import json
import time
import scipy.stats as stats
from numpy.random import MT19937, RandomState, SeedSequence

import matplotlib as mpl

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_probability as tp

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, cross_validate, RepeatedStratifiedKFold
from sklearn.metrics import median_absolute_error, mean_absolute_error, explained_variance_score, mean_squared_error, balanced_accuracy_score, roc_auc_score

import photonai
from photonai.base import Hyperpipe, PipelineElement, PhotonRegistry
from photonai.base.json_transformer import JsonTransformer
from photonai.processing import ResultsHandler
from photonai.processing.results_structure import MDBHyperpipe
from photonai.optimization import FloatRange, Categorical, IntegerRange

from Scripts.CustomCV import StratifiedGroupKFold, LeavePGroupsOut

# Ensure warnings are imported if used
import warnings

In [ ]:
print('photonai version: {}'.format(photonai.__version__))
print('tensorflow version: {}'.format(tf.__version__))
print('tensorflow-probability version: {}'.format(tp.__version__))

In [ ]:
# Set this paramater to 1 to avoid printing annoying warnings (default=0)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

# Configure GPU's here
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; #"0,1" - Which devices to use. Let us use a single GPU

# Optional - do NOT use any GPU's (only CPU)
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
print("Number of physical CPUs:", os.cpu_count())

tf.config.list_physical_devices('CPU')

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

tf.config.list_physical_devices('GPU')

### Load ENIGMA dataset

In [ ]:
SAVE_DIR = '/data/wbbruin/Desktop/ENIGMA_ANXIETY/cross_disorder_data_v3'

# This dataset contains duplicates of subjects that were included across multiple working groups (i.e. PD/SAD/GAD/SPH)
pooled_df = pd.read_csv(os.path.join(SAVE_DIR, 'pooled_cross_disorder_data.csv'))

# Here duplicates have been removed already. This dataset will be used for cross-disorder classifications
pooled_df_no_duplicates = pd.read_csv(os.path.join(SAVE_DIR, 'pooled_cross_disorder_data_no_duplicates.csv'))

pooled_df.shape, pooled_df_no_duplicates.shape

In [ ]:
# Apply age filter for adolescence: min = 10, max = 25 y/o

min_age, max_age = 10, 25

pooled_df = pooled_df.loc[(pooled_df.Age >= min_age) & (pooled_df.Age <= max_age)]

pooled_df_no_duplicates = pooled_df_no_duplicates.loc[(pooled_df_no_duplicates.Age >= min_age) & 
                                                      (pooled_df_no_duplicates.Age <= max_age)]

print(pooled_df.Age.min(), pooled_df.Age.max()) # Age range after filtering
pooled_df.shape, pooled_df_no_duplicates.shape

In [ ]:
pooled_df_no_duplicates.groupby(['WG', 'Dx']).size()

In [ ]:
# Extract labels for FreeSurfer columns 

first_idx = np.where(pooled_df_no_duplicates.columns.values == 'L_bankssts_surfavg')[0][0]
last_idx = np.where(pooled_df_no_duplicates.columns.values == 'ICV')[0][0]
FS_cols = pooled_df_no_duplicates.columns.values[first_idx:last_idx+1]

print("Total FS columns: {}".format(len(FS_cols)))

# Create a subset without global features (i.e. summarized measures over hemipsheres and ICV)
global_FS_features = ['LSurfArea', 'RSurfArea', 'LThickness', 'RThickness', 'ICV']
subset_mask = [f not in global_FS_features for f in FS_cols]
FS_cols_wo_global = FS_cols[subset_mask]

print("Total FS columns without global hemishpere measures and ICV: {}".format(len(FS_cols_wo_global)))

# Parse out different modalities (CT/CSA/SUBVOL)
ct_mask = ['thick' in f for f in FS_cols_wo_global]
csa_mask = ['surf' in f for f in FS_cols_wo_global]
subcort_mask = ~np.array(ct_mask) & ~np.array(csa_mask)

assert sum(ct_mask) + sum(csa_mask) + sum(subcort_mask) == len(FS_cols_wo_global)

In [ ]:
# Exclude subjects with too many missing features for pooled data across WG

def exclude_subjects_with_missing_features(df, FS_cols, completeness_threshold=0.75):
    # Extract FS features
    X = df[FS_cols].values

    N_features = len(FS_cols)

    # Create mask for subjects that have too many missing values
    N_missing_per_subject = np.sum(np.isnan(X), axis=1)
    p_missing_per_subject = N_missing_per_subject / float(N_features)
    p_missing_inclusion_mask = (p_missing_per_subject < (1 - completeness_threshold))
    n_missing_excluded = sum(~p_missing_inclusion_mask)

    print(f"{sum(N_missing_per_subject > 0)} of {len(N_missing_per_subject)} subjects have >=1 missing features")
    print(f"{n_missing_excluded} subjects excluded with >{round((1 - completeness_threshold) * 100)}% missing features")
    print(df.loc[~p_missing_inclusion_mask].groupby(['WG', 'Dx']).size())
    print()

    df = df.loc[p_missing_inclusion_mask]

    return df


pooled_df = exclude_subjects_with_missing_features(pooled_df, FS_cols_wo_global)

# Do the same for cross-disorder data without duplicate entries
pooled_df_no_duplicates = exclude_subjects_with_missing_features(pooled_df_no_duplicates, FS_cols_wo_global)

pooled_df.shape, pooled_df_no_duplicates.shape

### Print demographics for reporting

In [ ]:
pooled_df_no_duplicates.Sex.unique(), # And separately for 0 = Males / 1 = females

In [ ]:
pooled_df_no_duplicates.Dx.value_counts()

In [ ]:
pooled_df_no_duplicates.groupby(['WG', 'Dx']).size()

In [ ]:
pooled_df_no_duplicates.groupby(['WG', 'Dx']).Sex.mean()

In [ ]:
pooled_df_no_duplicates.groupby(['WG', 'Dx']).Age.mean()

In [ ]:
pooled_df_no_duplicates.groupby(['WG']).MultiSiteID.nunique()

In [ ]:
pooled_df_no_duplicates.shape

In [ ]:
len(pooled_df_no_duplicates.MultiSiteID.unique())

In [ ]:
pooled_df_no_duplicates.MultiSiteID

In [ ]:
# Only use healthy controls

HC_df = pooled_df_no_duplicates.loc[pooled_df_no_duplicates.Dx == 0]
HC_df.shape

In [ ]:
HC_df.groupby('Sex').Age.describe()

In [ ]:
# Extract X, y and groups in scikit-compatable format

# Check with experts: we now include global summary measures (ICV/total CT/SA). Is that OK?
X = HC_df[FS_cols].to_numpy() # FS_cols_wo_global
y = HC_df['Age'].to_numpy()
groups = HC_df['MultiSiteID'].to_numpy()

In [ ]:
len(np.unique(groups))

In [ ]:
# What is % missing data
np.isnan(X).sum() / X.size

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(pooled_df_no_duplicates.groupby(['MultiSiteID', 'WG', 'Dx']).size())

### Set up cross-validation strategy

In [ ]:
def plot_cv_strategy(cv_df):

    fig, ax = plt.subplots(dpi=200)
    le = LabelEncoder()
    site_nums = le.fit_transform(cv_df.SiteID)
    n_splits = len(cv_df.CV_test_iter.unique())

    # Sort cv_df
    sorted_cv_df = cv_df.sort_values(['SiteCount', 'CV_test_iter'], ascending=[False, False])
    ordered_idx = sorted_cv_df.ID.values

    for ii, (tr, tt) in enumerate(cv_splits):

        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(cv_df))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices[ordered_idx],
            marker="_",
            lw=10,
            cmap=plt.cm.coolwarm,
            vmin=-0.2,
            vmax=1.2,
        )

    # Sites
    ax.scatter(
        range(len(cv_df)), [ii + 1.5] * len(cv_df), 
        c=site_nums[ordered_idx], 
        marker="_", lw=10, cmap=plt.cm.prism
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["group"]
    ax.set(
        yticks=np.arange(n_splits + 1) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
    #     xlim=[0, 500],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)


In [ ]:
cv_strategy = 'StratifiedKFold'
# cv_strategy = 'StratifiedGroupKFold'
# cv_strategy = 'GroupKFold'

# First we create bins for age using qcut. We use q=10, this corresponds to deciles
age_binned, age_categories = pd.qcut(HC_df.Age, q=10, retbins=True, duplicates='drop')
age_binned_codes = age_binned.cat.codes.astype(str)

n_splits = 10
random_state = 0

# Save a dataframe for plotting/reporting
cv_df = pd.DataFrame(columns=['ID'], data=np.arange(len(HC_df)))
cv_df['Sex'] = HC_df.Sex.astype(int).values
cv_df['Age'] = HC_df.Age.values
cv_df['Age_bin'] = age_binned_codes.values
cv_df['CV_test_iter'] = np.nan
cv_df['labels_to_stratify'] = ''
cv_df['SiteID'] = HC_df.MultiSiteID.values
cv_df['SiteCount'] = cv_df.groupby('SiteID')['SiteID'].transform('count')

if cv_strategy == 'StratifiedKFold':

    # Here we will use age bins together with sex, site_ID and WG for stratification
    labels_to_stratify = list(zip(HC_df.WG, HC_df.MultiSiteID, HC_df.Sex.astype(int).astype(str), 
                                  age_binned_codes))

    labels_to_stratify = np.array(['_'.join(l) for l in labels_to_stratify])
    cv_df['labels_to_stratify'] = labels_to_stratify

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    cv_splits = []
    for i, (train, test) in enumerate(cv.split(y=labels_to_stratify, X=np.zeros_like(labels_to_stratify))):
        cv_splits.append((train, test)) 
        cv_df.iloc[test, cv_df.columns.get_loc('CV_test_iter')] = i
     
    
elif cv_strategy == 'StratifiedGroupKFold':

    # Here we will use age_bins together with sex for stratification (but groups are kept seperate!)
    labels_to_stratify = list(zip(HC_df.Sex.astype(int).astype(str), age_binned_codes))
    labels_to_stratify = np.array(['_'.join(l) for l in labels_to_stratify])
    cv_df['labels_to_stratify'] = labels_to_stratify
    
    cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    cv_splits = []
    for i, (train, test) in enumerate(cv.split(y=labels_to_stratify, X=np.zeros_like(HC_df.MultiSiteID),
                                               groups=HC_df.MultiSiteID)):
        cv_splits.append((train, test)) 
        cv_df.iloc[test, cv_df.columns.get_loc('CV_test_iter')] = i

        
elif cv_strategy == 'GroupKFold':
    
    cv = GroupKFold(n_splits=n_splits)
    labels_to_stratify = HC_df.MultiSiteID.values # This will be passed to Hyperpipe as 'groups'
    cv_df['labels_to_stratify'] = labels_to_stratify 
    
    cv_splits = []
    for i, (train, test) in enumerate(cv.split(y=np.zeros_like(HC_df.MultiSiteID), 
                                               X=np.zeros_like(HC_df.MultiSiteID),
                                               groups=HC_df.MultiSiteID)):
        cv_splits.append((train, test)) 
        cv_df.iloc[test, cv_df.columns.get_loc('CV_test_iter')] = i


# Visualize CV strategy
plot_cv_strategy(cv_df)

# Summarize info on test set
test_set_info = cv_df.copy().groupby('CV_test_iter')['Age', 'Sex'].mean().reset_index()
for i in cv_df.CV_test_iter.unique():
    test_set_info.loc[test_set_info.CV_test_iter == i, 'size'] = cv_df.loc[cv_df.CV_test_iter == i].shape[0]
    test_set_info.loc[test_set_info.CV_test_iter == i, 'n_sites'] = len(cv_df.loc[cv_df.CV_test_iter == i].SiteID.unique())
    test_set_info.loc[test_set_info.CV_test_iter == i, 'sites'] = ' '.join(cv_df.loc[cv_df.CV_test_iter == i].SiteID.unique())
    
test_set_info

In [ ]:
np.unique(cv_df.CV_test_iter)

### Set up MCCQRNN model

In [ ]:
base_folder = os.getcwd()
custom_elements_folder = os.path.join(base_folder, 'Scripts')

# Remove previously registered elements
registry = PhotonRegistry(custom_elements_folder=custom_elements_folder)
registry.delete('MccModel')
registry.delete('IterativeImputerTrees')

for path in os.listdir(custom_elements_folder):
    if path == '__pycache__':
        shutil.rmtree(os.path.join(custom_elements_folder, path))
    elif path == 'CustomElements.json':
        os.remove(os.path.join(custom_elements_folder, path))
        
cache_dir = os.path.join(base_folder, 'cache')
if os.path.isdir(cache_dir):
    shutil.rmtree(cache_dir)

In [ ]:
registry = PhotonRegistry(custom_elements_folder=custom_elements_folder)
registry.register(photon_name='MccModel',
                  class_str='MCCQRNN_Regressor.MCCQRNN_Regressor', element_type='Estimator')


In [ ]:
# Check registration

registry.info('MccModel')

In [ ]:
registry = PhotonRegistry(custom_elements_folder=custom_elements_folder)
registry.register(photon_name='MICE_IterativeImputer',
                  class_str='MICE_IterativeImputer.MICE_IterativeImputer', element_type='Transformer')

In [ ]:
# Check registration

registry.info('MICE_IterativeImputer')

In [ ]:
# Define pipeline

my_pipe = Hyperpipe('MCC_' + cv_strategy,
                    inner_cv = cv,                  
                    metrics=['mean_absolute_error', 'mean_squared_error', 
                             'pearson_correlation', 'explained_variance'],
                    best_config_metric='mean_absolute_error',
                    cache_folder='./cache/',
                    eval_final_performance=False,
                    verbosity=2,
                   )

my_pipe += PipelineElement('StandardScaler')

# my_pipe += PipelineElement('SimpleImputer')
my_pipe += PipelineElement('MICE_IterativeImputer')

my_pipe += PipelineElement('MccModel')

my_pipe

In [ ]:
# Comment if already ran

my_pipe.fit(data=X, targets=y, groups=labels_to_stratify) 

### Load fitted model, predictions, and metrics

In [ ]:
LOAD_RESULTS = True
print(cv_strategy)

path_mask = ['_'.join(['MCC', cv_strategy, 'results']) in path for path in os.listdir('.')]
result_dir_name = np.array(os.listdir('.'))[path_mask][0]
result_dir_path = os.path.join('.', result_dir_name)

if LOAD_RESULTS:
    # Use previously stored data
    result_json_path = os.path.join(result_dir_path, 'photon_result_file.json')
    json_file = json.load(open(result_json_path, 'r'))

    results = MDBHyperpipe.from_document(json_file)
    handler = ResultsHandler(results)

    # Load optimal Hyperpipe
    model_photon_path = os.path.join(result_dir_path, 'photon_best_model.photon')
    my_pipe = Hyperpipe.load_optimum_pipe(model_photon_path)
else:
    # Continue working with the results directly now
    handler = my_pipe.results_handler
    results = handler.results

print(result_dir_path)
my_pipe

### Parse results

In [ ]:
def pearson_correlation(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0][1]


def calculate_adj_BAG(df):
    """
    Calculate the uncertainty adjusted Brain-Age-Gap.

    Parameters:
    df (DataFrame): DataFrame containing 'y_pred', 'y_true', and 'std_aleatory_epistemic' columns.

    Returns:
    numpy.ndarray: Array of adjusted BAG HC scores.
    """
    
    assert np.all(df['y_pred'] == df['median_aleatory_epistemic'])
    
    adj_BAG = ((df['y_pred'] - df['y_true']) / df['std_aleatory_epistemic']).values
                  
    return adj_BAG


def calculate_metrics_across_folds(predictions_df):
    
    metrics_df = pd.DataFrame({'fold' : np.unique(predictions_df.fold.unique())})
    for i_fold in metrics_df.fold.values:

        y_pred_fold = predictions_df.loc[predictions_df.fold == i_fold].y_pred.values
        y_true_fold = predictions_df.loc[predictions_df.fold == i_fold].y_true.values

        metrics_df.loc[metrics_df.fold == i_fold,
                       'N'] = sum(predictions_df.fold == i_fold)
        metrics_df.loc[metrics_df.fold == i_fold, 
                       'MAE'] = mean_absolute_error(y_true=y_true_fold, y_pred=y_pred_fold)
        metrics_df.loc[metrics_df.fold == i_fold, 
                       'MedianAE'] = median_absolute_error(y_true=y_true_fold, y_pred=y_pred_fold)
        metrics_df.loc[metrics_df.fold == i_fold, 
                       'MSE'] = mean_squared_error(y_true=y_true_fold, y_pred=y_pred_fold)
        metrics_df.loc[metrics_df.fold == i_fold, 
                       'r'] = pearson_correlation(y_true=y_true_fold, y_pred=y_pred_fold)
        metrics_df.loc[metrics_df.fold == i_fold, 
                       'EV'] = explained_variance_score(y_true=y_true_fold, y_pred=y_pred_fold)
        metrics_df.loc[metrics_df.fold == i_fold, 
                       'rs'] = stats.spearmanr(y_true_fold, y_pred_fold)[0]

    mean, std = metrics_df.mean().values[2:].round(2), metrics_df.std().values[2:].round(2)
    metrics_df.loc[len(metrics_df)] = np.append(['mean', ''], mean)
    metrics_df.loc[len(metrics_df)] = np.append(['std', ''], std)
    
    return metrics_df

### Parse predictions and performance metrics for healthy controls

In [ ]:
# Load all predictions across CV iterations into a DataFrame (for healthy controls)
MCC_results_df = pd.DataFrame(handler.get_best_config_inner_fold_predictions())

# Extract folds used by photon-ai
inner_folds_preds = handler.get_best_config_inner_fold_predictions()

# Verify if the predictions match the expected values
assert np.all(MCC_results_df.y_true.values == np.array(inner_folds_preds['y_true'])), "Mismatch in y_true values with inner fold predictions"
assert np.all(MCC_results_df.y_true.values == y), "Mismatch in y_true values with y"
assert np.all(MCC_results_df.y_true.values == HC_df.Age), "Mismatch in y_true values with HC_df.Age"

# Ensure y_pred is not corrected for aleatory uncertainty by default
assert np.all(MCC_results_df['y_pred'] == MCC_results_df['median_noAleatory_epistemic']), "y_pred is not equal to median_noAleatory_epistemic"

# Replace y_pred with 'median_aleatory_epistemic'
MCC_results_df['y_pred'] = MCC_results_df['median_aleatory_epistemic']

# Calculate and add the adjusted BAG score
MCC_results_df['adj_BAG'] = calculate_adj_BAG(MCC_results_df)

In [ ]:
# Create DataFrame with metrics OVER folds and ACROSS folds.

metrics_df = calculate_metrics_across_folds(MCC_results_df)
metrics_df.to_csv(os.path.join(result_dir_path, 'metrics_on_training_HC.csv'))
metrics_df

### Now provide predictions for patients!

In [ ]:
# Prepare data for patients
PT_df = pooled_df_no_duplicates.loc[pooled_df_no_duplicates.Dx == 1]

# Extract X, y, and groups in scikit-compatible format
X_pt = PT_df[FS_cols].to_numpy()
y_pt = PT_df['Age'].to_numpy()
groups_pt = PT_df['MultiSiteID'].to_numpy()

# Print information about the dataset
print(f"Total samples: {pooled_df_no_duplicates.shape[0]}, Patients: {PT_df.shape[0]}, Healthy controls: {HC_df.shape[0]}")
print(f"Total sites: {len(pooled_df_no_duplicates.MultiSiteID.unique())}, Patient sites: {len(PT_df.MultiSiteID.unique())}, Healthy control sites: {len(HC_df.MultiSiteID.unique())}")

# Path for patient predictions
predictions_patients_csv_path = os.path.join(result_dir_path, 'predictions_patients.csv') # test

# Load or generate predictions for patients
if os.path.exists(predictions_patients_csv_path):
    predictions_patients_df = pd.read_csv(predictions_patients_csv_path)
else:
    y_pred_patients = my_pipe.predict(X_pt)
    
    # Store predictions for patients
    predictions_patients_df = pd.DataFrame(y_pred_patients)
    assert np.all(predictions_patients_df['y_pred'] == predictions_patients_df['median_noAleatory_epistemic']), "y_pred is not equal to median_noAleatory_epistemic"
    
    predictions_patients_df['y_pred'] = predictions_patients_df['median_aleatory_epistemic']
    predictions_patients_df['y_true'] = y_pt
    predictions_patients_df.to_csv(predictions_patients_csv_path, index=False)

# Verify y_pred and median_aleatory_epistemic match
assert np.all(predictions_patients_df['median_aleatory_epistemic'] == predictions_patients_df['y_pred']), "Mismatch between median_aleatory_epistemic and y_pred"

# Calculate and add the adjusted BAG score
predictions_patients_df['adj_BAG'] = calculate_adj_BAG(predictions_patients_df)

print(predictions_patients_df['adj_BAG'].mean())
predictions_patients_df.head()

In [ ]:
# Calculate and store metrics for patients (no cross-validation folds, so calculate over all patients)

# Extract true and predicted values
y_true = predictions_patients_df.y_true
y_pred = predictions_patients_df.y_pred

# Calculate metrics
mae = mean_absolute_error(y_true=y_true, y_pred=y_pred)
medae = median_absolute_error(y_true=y_true, y_pred=y_pred)
mse = mean_squared_error(y_true=y_true, y_pred=y_pred)
pearson_corr = pearson_correlation(y_true=y_true, y_pred=y_pred)
evs = explained_variance_score(y_true=y_true, y_pred=y_pred)

# Print the calculated metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Pearson Correlation: {pearson_corr}")
print(f"Explained Variance Score (EVS): {evs}")

In [ ]:
y_pred.shape, PT_df.shape

### Create plots (four panels) for Figure 1: model performance

In [ ]:
mpl.rcParams['figure.dpi'] = 300

# Plot performance for healthy controls (1A)

# Use scatterplot to color individual datapoints using 'hue'
g_scatter = sns.scatterplot(data=MCC_results_df, x="y_true", y="y_pred", 
                            hue = MCC_results_df.adj_BAG,
                            palette = sns.color_palette("PuOr_r", as_cmap=True)
                            )

colors = g_scatter.collections[0].properties()['facecolor']
plt.close()


min_, max_ = np.floor(MCC_results_df.y_true.min()), np.ceil(MCC_results_df.y_true.max())
g = sns.jointplot(data=MCC_results_df, x="y_true", y="y_pred", 
                  x_jitter=0,
                  kind="reg",
                  xlim=(min_ - 0.5 , max_ + 0.5),
                  joint_kws = {'scatter_kws' : dict(alpha=0.5, s=15),
                               'line_kws' : dict(color="darkred", alpha=0.5)},
                  marginal_kws={'color': 'darkgrey',
                               }
                 )

# Clear the axes containing the scatter plot
g.ax_joint.collections[0].set_visible(False)

#Plot each individual point separately
for i,row in enumerate(MCC_results_df[['y_true', 'y_pred']].values):
    g.ax_joint.plot(row[0], row[1], color=colors[i], marker='o', zorder=0, alpha=0.9)
    

# Plot text: avearge r2 (EV), pearson r, and MAE
rp_str = str(np.round(float(metrics_df.loc[metrics_df.fold == 'mean']['r'].values[0]), 2)).ljust(4 , '0')
EV_str = str(np.round(float(metrics_df.loc[metrics_df.fold == 'mean']['EV'].values[0]), 2)).ljust(4 , '0')
MAE_str = str(np.round(float(metrics_df.loc[metrics_df.fold == 'mean']['MAE'].values[0]), 2)).ljust(4 , '0')
rs_str = str(np.round(float(metrics_df.loc[metrics_df.fold == 'mean']['rs'].values[0]), 2)).ljust(4 , '0')

# g.fig.text
g.fig.text(0.15, 0.80, 
       f'$r_p$ = {rp_str}\n'
       f'$r_s$ = {rs_str}\n'
       f'EV = {EV_str}\n'
       f'MAE = {MAE_str}\n',
       va = 'top',
       fontsize=10)

g.set_axis_labels(xlabel='Chronological Age', ylabel='Predicted Brain Age', fontsize=10)
g.fig.set_figwidth(8)
g.fig.set_figheight(6)

g.savefig(os.path.join(result_dir_path, 'Figure_1A_performance_on_training_HC.png'))

# Extract ylim and yticks so we can set them in other figure
ax = g.fig.gca()
ylim = ax.get_ylim()
yticks = ax.get_yticks()

In [ ]:
# Plot performance for patients (1B)

# Use scatterplot to color individual datapoints using 'hue'
g_scatter = sns.scatterplot(data=predictions_patients_df, x="y_true", y="y_pred", 
                            hue = predictions_patients_df.adj_BAG,
                            palette = sns.color_palette("PuOr_r", as_cmap=True)
                            )

colors = g_scatter.collections[0].properties()['facecolor']
plt.close()


min_, max_ = np.floor(predictions_patients_df.y_true.min()), np.ceil(predictions_patients_df.y_true.max())

g = sns.jointplot(data=predictions_patients_df, x="y_true", y="y_pred", 
                  x_jitter=0,
                  kind="reg",
                  xlim=(min_ - 0.5 , max_ + 0.5),
                  ylim=ylim,
                  joint_kws = {'scatter_kws' : dict(alpha=0.5, s=15),
                               'line_kws' : dict(color="darkred", alpha=0.5)},
                  marginal_kws={'color': 'darkgrey',
                               }
                 )

# Clear the axes containing the scatter plot
g.ax_joint.collections[0].set_visible(False)

#Plot each individual point separately
for i,row in enumerate(predictions_patients_df[['y_true', 'y_pred']].values):
    g.ax_joint.plot(row[0], row[1], color=colors[i], marker='o', zorder=0, alpha=0.9)
    

# Plot text: avearge r2 (EV), pearson r, and MAE
rp_str = str(np.round(pearson_correlation(y_true=predictions_patients_df.y_true, 
                                         y_pred=predictions_patients_df.y_pred), 2)).ljust(4 , '0')
rs_str = str(np.round(stats.spearmanr(predictions_patients_df.y_true, 
                                      predictions_patients_df.y_pred), 2)[0]).ljust(4 , '0')
EV_str = str(np.round(explained_variance_score(y_true=predictions_patients_df.y_true, 
                                               y_pred=predictions_patients_df.y_pred), 2)).ljust(4 , '0')
MAE_str = str(np.round(mean_absolute_error(y_true=predictions_patients_df.y_true, 
                                           y_pred=predictions_patients_df.y_pred), 2)).ljust(4 , '0')

# g.fig.text
g.fig.text(0.15, 0.80, 
       f'$r_p$ = {rp_str}\n'
       f'$r_s$ = {rs_str}\n'
       f'EV = {EV_str}\n'
       f'MAE = {MAE_str}\n',
       va = 'top',
       fontsize=10)

g.set_axis_labels(xlabel='Chronological Age', ylabel='Predicted Brain Age', fontsize=10)
g.fig.set_figwidth(8)
g.fig.set_figheight(6)

g.savefig(os.path.join(result_dir_path, 'Figure_1B_performance_on_testing_patients.png'))

In [ ]:
# Plot BAG for healthy controls (1C)

# Calculate adjusted and unadjusted BAG
adj_BAG_HC = ((MCC_results_df['y_pred'] - MCC_results_df['y_true']) / MCC_results_df['std_aleatory_epistemic']).values
unadj_BAG_HC = (MCC_results_df['y_pred'] - MCC_results_df['y_true'])

# Print Pearson and Spearman correlations
print('Pearson correlations:')
print('adj', stats.pearsonr(np.array(MCC_results_df['y_true']), adj_BAG_HC))
print('unadj', stats.pearsonr(np.array(MCC_results_df['y_true']), unadj_BAG_HC))
print('Spearman correlations:')
print('adj', stats.spearmanr(np.array(MCC_results_df['y_true']), adj_BAG_HC))
print('unadj', stats.spearmanr(np.array(MCC_results_df['y_true']), unadj_BAG_HC))

# Create the figure and axes with desired width and height
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)

# Copy DataFrame for plotting
tmp_df = MCC_results_df.copy()
tmp_df['adj_BAG'] = adj_BAG_HC
tmp_df['unadj_BAG'] = unadj_BAG_HC

# Define min and max limits for x-axis
min_, max_ = np.floor(tmp_df.y_true.min()), np.ceil(tmp_df.y_true.max())

# Plot unadjusted BAG
sns.regplot(x='y_true', y='unadj_BAG', data=tmp_df, fit_reg=True, ax=ax,
            x_jitter=.0,
            line_kws=dict(alpha=0.75),
            scatter_kws=dict(alpha=0.2, s=15), label='Unadjusted')

slope_unadj, intercept_unadj, r_value, p_value, std_err = stats.linregress(x=ax.get_lines()[0].get_xdata(),
                                                                           y=ax.get_lines()[0].get_ydata())
slope_unadj = str(np.round(slope_unadj, 2))
intercept_unadj = str(np.round(intercept_unadj, 2))

# Plot adjusted BAG
sns.regplot(x='y_true', y='adj_BAG', data=tmp_df, fit_reg=True, ax=ax,
            x_jitter=.0,
            line_kws=dict(alpha=0.75),
            scatter_kws=dict(alpha=0.2, s=15), label='Uncertainty Adjusted')

slope_adj, intercept_adj, r_value, p_value, std_err = stats.linregress(x=ax.get_lines()[1].get_xdata(),
                                                                       y=ax.get_lines()[1].get_ydata())
slope_adj = str(np.round(slope_adj, 2))
intercept_adj = str(np.round(intercept_adj, 2))

# Set labels and limits
ax.set(ylabel='Brain Age Gap', xlabel='Chronological Age')
ax.set_xlim(min_ - 0.5, max_ + 0.5)
ax.legend(loc='upper right')

# Calculate and format correlation values
rp_unadj_str = str(np.round(stats.pearsonr(tmp_df.y_true, tmp_df.unadj_BAG)[0], 2))
rs_unadj_str = str(np.round(stats.spearmanr(tmp_df.y_true, tmp_df.unadj_BAG)[0], 2))

rp_adj_str = str(np.round(stats.pearsonr(tmp_df.y_true, tmp_df.adj_BAG)[0], 2))
rs_adj_str = str(np.round(stats.spearmanr(tmp_df.y_true, tmp_df.adj_BAG)[0], 2))

# Add text boxes with regression info
fig.text(0.15, 0.25,
         'Unadjusted \n'
         f'y = {intercept_unadj} + {slope_unadj}x\n'
         f'$r_p$ = {rp_unadj_str}\n'
         f'$r_s$ = {rs_unadj_str}\n',
         va='top',
         fontsize=10)

fig.text(0.35, 0.25,
         'Adjusted \n'
         f'y = {intercept_adj} + {slope_adj}x\n'
         f'$r_p$ = {rp_adj_str}\n'
         f'$r_s$ = {rs_adj_str}\n',
         va='top',
         fontsize=10)

# Set fontsize for labels
ax.tick_params(axis='both', which='major', labelsize=10)

# Save and show the plot
plt.savefig(os.path.join(result_dir_path, 'Figure_1C_BAG_for_healthy_controls.png'))
plt.show()

In [ ]:
# Plot BAG for patients (1D)

# Calculate adjusted and unadjusted BAG
adj_BAG_pred_patients = ((predictions_patients_df['y_pred'] - predictions_patients_df['y_true']) / predictions_patients_df['std_aleatory_epistemic']).values
unadj_BAG_pred_patients = (predictions_patients_df['y_pred'] - predictions_patients_df['y_true'])

# Print Pearson and Spearman correlations
print('Pearson correlations:')
print('adj', stats.pearsonr(np.array(predictions_patients_df['y_true']), adj_BAG_pred_patients))
print('unadj', stats.pearsonr(np.array(predictions_patients_df['y_true']), unadj_BAG_pred_patients))
print('Spearman correlations:')
print('adj', stats.spearmanr(np.array(predictions_patients_df['y_true']), adj_BAG_pred_patients))
print('unadj', stats.spearmanr(np.array(predictions_patients_df['y_true']), unadj_BAG_pred_patients))

# Create the figure and axes with desired width and height
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)

# Copy DataFrame for plotting
tmp_df = predictions_patients_df.copy()
tmp_df['adj_BAG'] = adj_BAG_pred_patients
tmp_df['unadj_BAG'] = unadj_BAG_pred_patients

# Define min and max limits for x-axis
min_, max_ = np.floor(tmp_df.y_true.min()), np.ceil(tmp_df.y_true.max())

# Plot unadjusted BAG
sns.regplot(x='y_true', y='unadj_BAG', data=tmp_df, fit_reg=True, ax=ax,
            x_jitter=.0,
            line_kws=dict(alpha=0.75),
            scatter_kws=dict(alpha=0.2, s=15), label='Unadjusted')

slope_unadj, intercept_unadj, r_value, p_value, std_err = stats.linregress(x=ax.get_lines()[0].get_xdata(),
                                                                           y=ax.get_lines()[0].get_ydata())
slope_unadj = str(np.round(slope_unadj, 2))
intercept_unadj = str(np.round(intercept_unadj, 2))

# Plot adjusted BAG
sns.regplot(x='y_true', y='adj_BAG', data=tmp_df, fit_reg=True, ax=ax,
            x_jitter=.0,
            line_kws=dict(alpha=0.75),
            scatter_kws=dict(alpha=0.2, s=15), label='Uncertainty Adjusted')

slope_adj, intercept_adj, r_value, p_value, std_err = stats.linregress(x=ax.get_lines()[1].get_xdata(),
                                                                       y=ax.get_lines()[1].get_ydata())
slope_adj = str(np.round(slope_adj, 2))
intercept_adj = str(np.round(intercept_adj, 2))

# Set labels and limits
ax.set(ylabel='Brain Age Gap', xlabel='Chronological Age')
ax.set_xlim(min_ - 0.5, max_ + 0.5)
ax.legend(loc='upper right')

# Calculate and format correlation values
rp_unadj_str = str(np.round(stats.pearsonr(tmp_df.y_true, tmp_df.unadj_BAG)[0], 2))
rs_unadj_str = str(np.round(stats.spearmanr(tmp_df.y_true, tmp_df.unadj_BAG)[0], 2))

rp_adj_str = str(np.round(stats.pearsonr(tmp_df.y_true, tmp_df.adj_BAG)[0], 2))
rs_adj_str = str(np.round(stats.spearmanr(tmp_df.y_true, tmp_df.adj_BAG)[0], 2))

# Add text boxes with regression info
fig.text(0.15, 0.25,
         'Unadjusted \n'
         f'y = {intercept_unadj} + {slope_unadj}x\n'
         f'$r_p$ = {rp_unadj_str}\n'
         f'$r_s$ = {rs_unadj_str}\n',
         va='top',
         fontsize=10)

fig.text(0.35, 0.25,
         'Adjusted \n'
         f'y = {intercept_adj} + {slope_adj}x\n'
         f'$r_p$ = {rp_adj_str}\n'
         f'$r_s$ = {rs_adj_str}\n',
         va='top',
         fontsize=10)

# Set fontsize for labels
ax.tick_params(axis='both', which='major', labelsize=10)

# Save and show the plot
plt.savefig(os.path.join(result_dir_path, 'Figure_1D_BAG_for_patients.png'))
plt.show()

### Bias Assessment

In [ ]:
# Calculate MAE over all subjects
overall_mae = mean_absolute_error(y_true=MCC_results_df.y_true.values, 
                                  y_pred=MCC_results_df.y_pred.values)
print(f"Overall MAE: {overall_mae:.2f}")

# Standardized MAE
standardized_mae = overall_mae / np.std(HC_df.Age)
print(f"Standardized MAE: {standardized_mae:.2f}")
print()

# Calculate SMAE separately for healthy controls
sex = {0: 'males', 1: 'females'}

for c, gender in sex.items():
    print(f"Healthy Controls - {gender}:")
    mask = HC_df.Sex == c
    mae = mean_absolute_error(y_true=np.array(MCC_results_df.y_true.values)[mask], 
                              y_pred=np.array(MCC_results_df.y_pred.values)[mask])
    smae = mae / np.std(HC_df.Age[mask])
    print(f'MAE: {mae:.2f}')
    print(f'SMAE: {smae:.2f}')
    print()

# Calculate SMAE separately for patients
for c, gender in sex.items():
    print(f"Patients - {gender}:")
    mask = PT_df.Sex == c
    mae = mean_absolute_error(y_true=np.array(predictions_patients_df.y_true.values)[mask], 
                              y_pred=np.array(predictions_patients_df.y_pred.values)[mask])
    smae = mae / np.std(PT_df.Age[mask])
    print(f'MAE: {mae:.2f}')
    print(f'SMAE: {smae:.2f}')
    print()


### Subject-wise probability of accelerated brain aging 

In [ ]:
# Calculate subject-wise probability of accelerated brain aging: first quantile containing true age of subject

def calculate_quantile_indices_and_scores(df):
    quantile_indices = []
    alpha_scores = []
    quantiles = [f'{frac:.3f}_aleatory_epistemic' for frac in np.arange(100 + 1) * 0.01]

    for i, row in df.iterrows():
        quantile_idx = np.where(row[quantiles].values > row.y_true)[0]

        if len(quantile_idx) == 0:
            # Append 100 if BAG is negative, append 0 if BAG is positive
            if row.adj_BAG > 0:
                quantile_indices.append(0)
                alpha_scores.append(50)
            else:
                quantile_indices.append(100)
                alpha_scores.append(-50)
        else:
            quantile_idx = quantile_idx[0]
            quantile_indices.append(quantile_idx)
            quantile_idx = 50 - quantile_idx if quantile_idx < 50 else -1 * (quantile_idx - 50)
            alpha_scores.append(quantile_idx)

    df['quantile_indices'] = quantile_indices
    df['alpha_scores'] = alpha_scores
    df['alpha_p'] = (50 - df.quantile_indices) / 50
    return df

def get_extreme_bag_examples(df, N=1):
    unique_alpha_scores = np.sort(df['alpha_scores'].unique())
    highest_bag_scores = unique_alpha_scores[-N:]
    lowest_bag_scores = unique_alpha_scores[:N]

    highest_bag = df[df['alpha_scores'].isin(highest_bag_scores)]
    lowest_bag = df[df['alpha_scores'].isin(lowest_bag_scores)]

    highest_bag = highest_bag.groupby('alpha_scores').apply(lambda x: x.sample(1)).reset_index(drop=True)
    lowest_bag = lowest_bag.groupby('alpha_scores').apply(lambda x: x.sample(1)).reset_index(drop=True)
    
    return highest_bag, lowest_bag

# Calculate for patients
predictions_patients_df = calculate_quantile_indices_and_scores(predictions_patients_df)
highest_patient, lowest_patient = get_extreme_bag_examples(predictions_patients_df, 3)

# Calculate for healthy controls
MCC_results_df = calculate_quantile_indices_and_scores(MCC_results_df)
highest_hc, lowest_hc = get_extreme_bag_examples(MCC_results_df, 3)

# Printing example highest and lowest BAG for patients and healthy controls

cols_to_print = ['0.000_aleatory_epistemic', '0.010_aleatory_epistemic', 
                 '0.990_aleatory_epistemic', '1.000_aleatory_epistemic', 
                 'y_pred', 'y_true', 
                 'adj_BAG', 'quantile_indices', 'alpha_scores', 'alpha_p']

print("Example highest BAG for patients:")
print(display(highest_patient[cols_to_print]))
print("\n\nExample lowest BAG for patients:")
print(display(lowest_patient[cols_to_print]))


print("\n\nExample highest BAG for healthy controls:")
print(display(highest_hc[cols_to_print]))
print("\n\nExample lowest BAG for healthy controls:")
print(display(lowest_hc[cols_to_print]))

In [ ]:
# Combine predictionss
predictions_patients_df['group'] = 'Anxiety Disorder'
MCC_results_df['group'] = 'Healthy Control'
combined_df = pd.concat([predictions_patients_df, MCC_results_df])

# Calculate means for each group
mean_patient = combined_df.loc[combined_df['group'] == 'Anxiety Disorder', 'adj_BAG'].mean()
mean_control = combined_df.loc[combined_df['group'] == 'Healthy Control', 'adj_BAG'].mean()

# Set up the plot
plt.figure(figsize=(20, 6))

# Plot the KDE for both groups and fill the area under the curve
sns.kdeplot(data=combined_df[combined_df['group'] == 'Anxiety Disorder'], 
            x='adj_BAG', fill=True, color='Red', alpha=0.4, label='Anxiety Disorder')
sns.kdeplot(data=combined_df[combined_df['group'] == 'Healthy Control'], 
            x='adj_BAG', fill=True, color='Blue', alpha=0.4, label='Healthy Control')

# Add vertical lines for the mean values
plt.axvline(mean_patient, color='Red', linestyle='--', linewidth=1.5)  # Match color for patient group
plt.axvline(mean_control, color='Blue', linestyle='--', linewidth=1.5)  # Match color for control group

# Add titles and labels
plt.title('Distribution of BAG scores for Anxiety Disorder Patients and Healthy Controls')
plt.xlabel('Uncertainty-Adjusted BAG')
plt.ylabel('Density')
# plt.legend(title='Group')  # Add legend to show mean lines

plt.xlim(-2.5, 2.5)

# Show the plot
plt.show()

In [ ]:
# Combine the data
combined_df = pd.read_csv('BAG_residuals_per_group.csv')
combined_df['group'] = combined_df['Dx'].map({0: 'Healthy Control', 1: 'Anxiety Disorder'})

# Calculate means for each group
mean_patient = combined_df.loc[combined_df['group'] == 'Anxiety Disorder', 'plot_res'].mean()
mean_control = combined_df.loc[combined_df['group'] == 'Healthy Control', 'plot_res'].mean()

# Set up the plot
plt.figure(figsize=(20, 6))

# Plot the KDE for both groups and fill the area under the curve
sns.kdeplot(data=combined_df[combined_df['group'] == 'Anxiety Disorder'], 
            x='plot_res', fill=True, color='Red', alpha=0.4, label='Anxiety Disorder')
sns.kdeplot(data=combined_df[combined_df['group'] == 'Healthy Control'], 
            x='plot_res', fill=True, color='Blue', alpha=0.4, label='Healthy Control')

# Add vertical lines for the mean values
plt.axvline(mean_patient, color='Red', linestyle='--', linewidth=1.5)  # Match color for patient group
plt.axvline(mean_control, color='Blue', linestyle='--', linewidth=1.5)  # Match color for control group

# Add titles and labels
plt.title('Distribution of BAG scores for Anxiety Disorder Patients and Healthy Controls')
plt.xlabel('Uncertainty-Adjusted BAG (residuals)')
plt.ylabel('Density')
plt.legend(title='Group')  # Add legend to show mean lines

plt.xlim(-2.5, 2.5)

# Show the plot
plt.show()

### Occlusion sensitivity mapping

In [ ]:
# Define paths and variables
occlusion_sensitivity_mapping_dir = os.path.join(result_dir_path, 'occlusion_sensitivity_mapping') 

occlusion_adj_BAG_patients_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_adj_BAG_patients.npy')
occlusion_median_aleatory_epistemic_patients_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_median_aleatory_epistemic_patients.npy')
occlusion_std_aleatory_epistemic_patients_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_std_aleatory_epistemic_patients.npy')
original_adj_BAG_patients_path = os.path.join(occlusion_sensitivity_mapping_dir, 'original_adj_BAG_patients.npy')
predictions_patients_csv_path = os.path.join(occlusion_sensitivity_mapping_dir, 'predictions_patients.csv')

# Check if results directory exists
if all([os.path.exists(occlusion_adj_BAG_patients_path),
        os.path.exists(occlusion_median_aleatory_epistemic_patients_path),
        os.path.exists(occlusion_std_aleatory_epistemic_patients_path),
        os.path.exists(original_adj_BAG_patients_path),
        os.path.exists(predictions_patients_csv_path)]):
    
    print('Loading results')
    
    occlusion_adj_BAG_patients = np.load(occlusion_adj_BAG_patients_path, allow_pickle=True)
    occlusion_median_aleatory_epistemic_patients = np.load(occlusion_median_aleatory_epistemic_patients_path, allow_pickle=True)
    occlusion_std_aleatory_epistemic_patients = np.load(occlusion_std_aleatory_epistemic_patients_path, allow_pickle=True)
    adj_BAG_pred_patients = np.load(original_adj_BAG_patients_path, allow_pickle=True)
    predictions_patients_df = pd.read_csv(predictions_patients_csv_path)
    
else:
    
    # Results directory does not exist, perform occlusion sensitivity mapping
    print('Performing occlusion sensitivity mapping')
    
    # Calculate number of patients and number of features
    N_patients, N_features = X_pt.shape
    
    # Check if the directory exists, if not, create it
    if not os.path.exists(occlusion_sensitivity_mapping_dir):
        os.makedirs(occlusion_sensitivity_mapping_dir)
    
    adj_BAG_pred_patients = predictions_patients_df.adj_BAG.values
    
    # Create occlusion FS data and initialize arrays for occlusion results
    occlusion_FS_data_patients = np.repeat(X_pt[:, :, np.newaxis], N_features, axis=2)
    occlusion_adj_BAG_patients = np.zeros((N_patients, N_features))
    occlusion_median_aleatory_epistemic_patients = np.zeros_like(occlusion_adj_BAG_patients)
    occlusion_std_aleatory_epistemic_patients = np.zeros_like(occlusion_adj_BAG_patients)
    
    # Perform occlusion sensitivity mapping
    for occlusion_idx in np.arange(N_features):
        occlusion_FS_data_patients[:, occlusion_idx, occlusion_idx] = 0 # Occlude ROI
        tmp_y_pred = my_pipe.predict(occlusion_FS_data_patients[:, :, occlusion_idx])
        tmp_adj_BAG = (tmp_y_pred['median_aleatory_epistemic'] - predictions_patients_df['y_true'] ) / tmp_y_pred['std_aleatory_epistemic']
        
        occlusion_adj_BAG_patients[:, occlusion_idx] = tmp_adj_BAG
        occlusion_median_aleatory_epistemic_patients[:, occlusion_idx] = tmp_y_pred['median_aleatory_epistemic']
        occlusion_std_aleatory_epistemic_patients[:, occlusion_idx] = tmp_y_pred['std_aleatory_epistemic']
    
    # Save results
    np.save(occlusion_adj_BAG_patients_path, occlusion_adj_BAG_patients, allow_pickle=True)
    np.save(occlusion_median_aleatory_epistemic_patients_path, occlusion_median_aleatory_epistemic_patients, allow_pickle=True)
    np.save(occlusion_std_aleatory_epistemic_patients_path, occlusion_std_aleatory_epistemic_patients, allow_pickle=True)
    np.save(original_adj_BAG_patients_path, adj_BAG_pred_patients, allow_pickle=True)
    predictions_patients_df.to_csv(predictions_patients_csv_path, index=False)


In [ ]:
# Now do the same for healthy controls

occlusion_adj_BAG_HC_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_adj_BAG_HC.npy')
occlusion_median_aleatory_epistemic_HC_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_median_aleatory_epistemic_HC.npy')
occlusion_std_aleatory_epistemic_HC_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_std_aleatory_epistemic_HC.npy')
original_adj_BAG_HC_path = os.path.join(occlusion_sensitivity_mapping_dir, 'original_adj_BAG_HC.npy')
predictions_HC_csv_path = os.path.join(occlusion_sensitivity_mapping_dir, 'predictions_HC.csv')

# Check if results directory exists
if all([os.path.exists(occlusion_adj_BAG_HC_path),
        os.path.exists(occlusion_median_aleatory_epistemic_HC_path),
        os.path.exists(occlusion_std_aleatory_epistemic_HC_path),
        os.path.exists(original_adj_BAG_HC_path),
        os.path.exists(predictions_HC_csv_path)]):
    
    print('Loading results')
    
    occlusion_adj_BAG_HC = np.load(occlusion_adj_BAG_HC_path, allow_pickle=True)
    occlusion_median_aleatory_epistemic_HC = np.load(occlusion_median_aleatory_epistemic_HC_path, allow_pickle=True)
    occlusion_std_aleatory_epistemic_HC = np.load(occlusion_std_aleatory_epistemic_HC_path, allow_pickle=True)
    adj_BAG_pred_HC = np.load(original_adj_BAG_HC_path, allow_pickle=True)
    MCC_results_df = pd.read_csv(predictions_HC_csv_path)
    
else:
    
    # Results directory does not exist, perform occlusion sensitivity mapping
    print('Performing occlusion sensitivity mapping')
    
    # Calculate number of HC and number of features
    X_hc = HC_df[FS_cols].to_numpy()
    N_HC, N_features = X_hc.shape
    
    # Check if the directory exists, if not, create it
    if not os.path.exists(occlusion_sensitivity_mapping_dir):
        os.makedirs(occlusion_sensitivity_mapping_dir)
    
    adj_BAG_pred_HC = MCC_results_df.adj_BAG.values
    
    # Create occlusion FS data and initialize arrays for occlusion results
    occlusion_FS_data_HC = np.repeat(X_hc[:, :, np.newaxis], N_features, axis=2)
    occlusion_adj_BAG_HC = np.zeros((N_HC, N_features))
    occlusion_median_aleatory_epistemic_HC = np.zeros_like(occlusion_adj_BAG_HC)
    occlusion_std_aleatory_epistemic_HC = np.zeros_like(occlusion_adj_BAG_HC)
    
    # Perform occlusion sensitivity mapping
    for occlusion_idx in np.arange(N_features):
        occlusion_FS_data_HC[:, occlusion_idx, occlusion_idx] = 0 # Occlude ROI
        tmp_y_pred = my_pipe.predict(occlusion_FS_data_HC[:, :, occlusion_idx])
        tmp_adj_BAG = (tmp_y_pred['median_aleatory_epistemic'] - MCC_results_df['y_true']) / tmp_y_pred['std_aleatory_epistemic']
        
        occlusion_adj_BAG_HC[:, occlusion_idx] = tmp_adj_BAG
        occlusion_median_aleatory_epistemic_HC[:, occlusion_idx] = tmp_y_pred['median_aleatory_epistemic']
        occlusion_std_aleatory_epistemic_HC[:, occlusion_idx] = tmp_y_pred['std_aleatory_epistemic']
    
    # Save results
    np.save(occlusion_adj_BAG_HC_path, occlusion_adj_BAG_HC, allow_pickle=True)
    np.save(occlusion_median_aleatory_epistemic_HC_path, occlusion_median_aleatory_epistemic_HC, allow_pickle=True)
    np.save(occlusion_std_aleatory_epistemic_HC_path, occlusion_std_aleatory_epistemic_HC, allow_pickle=True)
    np.save(original_adj_BAG_HC_path, adj_BAG_pred_HC, allow_pickle=True)
    MCC_results_df.to_csv(predictions_HC_csv_path, index=False)


#### Store occlusion sensitivity data in long format for R studio analyses

In [ ]:
# Create dataframe that includes all variables used to fit LME model. 

occlusion_adj_BAG_scores_df = pd.DataFrame(data=occlusion_adj_BAG_patients, columns=FS_cols)
occlusion_adj_BAG_scores_df['no_occlusion'] = adj_BAG_pred_patients
controlling_vars = PT_df[['SubjID', 'Age', 'Sex', 'MultiSiteID']].copy().reset_index()

data_df = occlusion_adj_BAG_scores_df.join(controlling_vars)
data_df = data_df.drop('index', axis=1)

assert np.all(data_df.Age.values == y_pt)
assert np.all(data_df.MultiSiteID.values == groups_pt)
assert np.all(data_df.no_occlusion.values == adj_BAG_pred_patients)

data_df = pd.melt(data_df, id_vars=['SubjID', 'Age', 'Sex', 'MultiSiteID'], 
                  value_vars=occlusion_adj_BAG_scores_df.columns.values,
                  value_name='BAGz',
                  var_name='region')


long_data_csv_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_data_long_MCCQRNN_Regressor.csv')
data_df.to_csv(long_data_csv_path)

data_df.head()

In [ ]:
# Do the same for HC: load results - save as long format

occlusion_adj_BAG_scores_df = pd.DataFrame(data=occlusion_adj_BAG_HC, columns=FS_cols)
occlusion_adj_BAG_scores_df['no_occlusion'] = adj_BAG_pred_HC
controlling_vars = HC_df[['SubjID', 'Age', 'Sex', 'MultiSiteID']].copy().reset_index()

data_df = occlusion_adj_BAG_scores_df.join(controlling_vars)
data_df = data_df.drop('index', axis=1)

assert np.all(data_df.no_occlusion.values == adj_BAG_pred_HC)

data_df = pd.melt(data_df, id_vars=['SubjID', 'Age', 'Sex', 'MultiSiteID'], 
                  value_vars=occlusion_adj_BAG_scores_df.columns.values,
                  value_name='BAGz',
                  var_name='region')


long_data_csv_path = os.path.join(occlusion_sensitivity_mapping_dir, 'occlusion_data_HC_long_MCCQRNN_Regressor.csv')
data_df.to_csv(long_data_csv_path)

data_df.head()